In [1]:
# 자소 분리 및 결합 코드
cho_list = ['ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
jung_list = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ',
             'ㅣ']
jong_list = ['', 'ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 'ㅄ', 'ㅅ',
             'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']

def jaso_split(korean_word):
    result = []
    for c in list(korean_word.strip()):
        # 현재 문자가 한글일 경우
        if '가' <= c <= '힣':
            chosung = (ord(c) - ord('가')) // 588
            jungsung = ((ord(c) - ord('가')) - (588 * chosung)) // 28
            jongsung = (ord(c) - ord('가')) - (588 * chosung) - (28 * jungsung)
            result.append(cho_list[chosung])
            result.append(jung_list[jungsung])
            if jongsung:
                result.append(jong_list[jongsung])
        # 현재 문자가 한글이 아닐 경우
        else:
            result.append(c)

    return result

def jaso_combine(chosung, jungsung, jongsung=''):
    if chosung and jungsung:
        result = chr((28 * int(jung_list.index(jungsung))) + int(jong_list.index(jongsung)) + (588 * cho_list.index(chosung)) + ord('가'))
    else:
        result = ''.join([chosung, jungsung, jongsung])
    return result

In [2]:
from itertools import chain, combinations, product

def viewTable(table, n):
  #테이블 내부 원소 확인하는 함수
  for i in table:
      for j in i:
        print(j, end=' ')
      print()

def extractionList(table):
  # 문자열과 딕셔너리 key를 추출하여 조합한 결과를 저장할 리스트
  result_list = []
  # 리스트를 순회하면서 처리
  for item in table:
      if isinstance(item, str):  # 문자열인 경우
          result_list.append(item)
      elif isinstance(item, dict):  # 딕셔너리인 경우
          result_list.extend(item.keys())
      elif isinstance(item, list):  # 리스트인 경우
          if item:
              first_dict = item[0]  # 첫 번째 원소의 딕셔너리를 추출합니다.
              for key in first_dict.keys():  # 첫 번째 원소의 key를 추출하여 결과 리스트에 추가합니다.
                  result_list.append(key)

  result_string = ''.join(result_list)
  return result_string

def extract_keys(input_list):
  #주어진 리스트 내의 딕셔너리에서 key 추출하는 함수
    result = []
    for element in input_list:
        if isinstance(element, dict):
            # 딕셔너리인 경우 key를 추출하여 결과 리스트에 추가
            result.extend(element.keys())
        elif isinstance(element, list):
            # 리스트인 경우 첫 번째 원소의 key를 추출하여 결과 리스트에 추가
            if isinstance(element[0], dict):
                result.extend(element[0].keys())
    return result

def extract_sublists(input_list):
  #주어진 리스트 내의 부분 리스트 추출하는 함수
    sublists = []
    for i in range(len(input_list)):
        for j in range(i + 1, len(input_list) + 1):
            sublists.append(input_list[i:j])
    return sublists

def match_grammar(matched_words, grammar):
    matched_combinations = []
    flag = True

    # 주어진 리스트와 딕셔너리의 모든 조합을 확인해서 접속 검사
    for key1, value1 in matched_words.items():
        for key2, value2 in matched_words.items():
            if key1 != key2:
                # 모든 가능한 품사 조합을 생성
                for pos1, pos2 in product(value1, value2):
                    if key1 == input_text and flag:
                        matched_combinations.append([key1+'/'+pos1])
                        flag=False
                        break
                    pos_combination = pos1 + '+' + pos2
                    if pos_combination in grammar:
                        combined_word = key1 + key2
                        # 조합된 단어가 주어진 길이와 같으면 추가
                        if combined_word == input_text:
                            # 순서를 유지하면서 조합을 추가
                            matched_combinations.append([key1 + '/' + pos1, key2 + '/' + pos2])

    # 하나의 키-값 쌍만 존재하는 경우 해당 값을 그대로 추가
    if len(matched_words) == 1:
        for key, value in matched_words.items():
            matched_combinations.append([key + '/' + pos for pos in value])

    return matched_combinations

In [3]:
def tabular_parsing(jaso_list, morpheme, grammar):
  n = len(jaso_list)
  #테이블 생성
  table = [[[] for _ in range(n)] for _ in range(n)]

  # 테이블의 0열을 자소로 초기화
  for i in range(n):
      table[i][0] = [jaso_list[n - i - 1]]
  cho=''
  jung=''
  jong=''
  temp2=''

  # 매커니즘에 따라 테이블 채우기
  for i in range(1, n):
    for j in range(1, i + 1):
      # j열의 값을 구하기 위해 j-1열과 0열부터 i-j행까지의 값을 이용하여 결합합니다.
      table[i][j] = table[i][j - 1] + table[i - j][0]

      for k in range(len(table[i][j])):#사전 검색
        if table[i][j][k] in cho_list:#해당 자소가 초성인 경우
          cho=table[i][j][k]
        if k<len(table[i][j])-1 and table[i][j][k+1] in jung_list:#중성이 있고
          if k<len(table[i][j])-2 and table[i][j][k+2] in jong_list:#종성도 있는 경우
            if k<len(table[i][j])-3 and table[i][j][k+3] in jung_list:#종성 다음이 중성이 있는 경우
              break
            #종성 다음에 중성이 없는 경우 3가지 자소
            jong = table[i][j][k+2]
            jung = table[i][j][k+1]
            temp = jaso_combine(cho, jung,jong)
            temp2+=temp
            if temp2 in morpheme: #temp2가 자소 set의 사전 검색이 가능하면
              table[i][j].remove(cho)
              table[i][j].remove(jung)
              table[i][j].remove(jong)
              if temp2 in table[i][j]:#temp2가 테이블에 저장되어 있다면
                table[i][j].remove(temp2)#없애고
              for l in morpheme[temp2]:
                table[i][j].append({temp2:l})#사전 할당
              temp2=''
            else:#해당 자소 set이 사전 검색이 안되면 일단 그것은 backup후 나중에 체크
              table[i][j].remove(cho)
              table[i][j].remove(jung)
              table[i][j].remove(jong)
              if temp2 == '':
                temp2 = temp
                if temp2 in table[i][j]:
                  table[i][j].append(temp)
              else:
                if temp2 in table[i][j]:
                  table[i][j].remove(temp2)
                  temp2+=temp
                  table[i][j].append(temp2)
            break

          else:
            #종성이 없는 경우 2가지 자소
            jung=table[i][j][k+1]
            temp = jaso_combine(cho, jung)
            if temp in morpheme:
              table[i][j].remove(cho)
              table[i][j].remove(jung)
              t = []
              for l in morpheme[temp]:
                t.append({temp:l})
              table[i][j].append(t)
              break
      if i==j:
          for k in range(n):
            if table[k][0][0] in jung_list:
              continue
            else:
              my_list = extract_sublists(extract_keys(table[i][j]))
              for i in range(len(my_list)):
                my_list[i] = ''.join(my_list[i])
              # 사전 검사 후 접속검사에 match할 것들을 담을 딕셔너리
              matched_words = {}
              for word in my_list:
                  if word in morpheme:
                      matched_words[word] = morpheme[word]
              matched_combinations = match_grammar(matched_words, grammar)
              break

  result = sorted(matched_combinations)
  return result

In [12]:
if __name__ == '__main__':
    morpheme = {"건국": ["NNG"], "국": ["NNG"], "대": ["NNB"], "대가": ["NNG"], "건국대": ["NNP"], "가": ["NNG", "JKS"]}
    grammar = ["NNG+NNG", "NNB+JKS", "NNG+JKS", "NNP+NNG", "NNP+JKS"]

    # input_text = input("Input Sentence : ")
    input_text = "건국대가"
    jaso_list = jaso_split(input_text)
    print(jaso_list)
    print(tabular_parsing(jaso_list, morpheme, grammar))


['ㄱ', 'ㅓ', 'ㄴ', 'ㄱ', 'ㅜ', 'ㄱ', 'ㄷ', 'ㅐ', 'ㄱ', 'ㅏ']
[['건국/NNG', '대가/NNG'], ['건국대/NNP', '가/JKS'], ['건국대/NNP', '가/NNG']]
